In [1]:
import splitfolders

splitfolders.ratio("E:\PlantVillage", 
                   output="train_data", 
                   seed=42, 
                   ratio=(.75, .15, .1), 
                   group_prefix=None, 
                   move=False 
                   )

Copying files: 18164 files [06:48, 44.51 files/s]


In [1]:
import numpy as np
import time
import copy
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

In [2]:
#transforms
transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [3]:
data_dir = 'E:/train_data'

In [4]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=transforms[x])
                  for x in ['train', 'val', 'test']}

In [5]:

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=2, shuffle=True, num_workers=2)
               for x in ['train', 'val', 'test']}

In [6]:
data_size = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

In [7]:
data_size

{'train': 13619, 'val': 2719, 'test': 1825}

In [8]:
class_names = image_datasets['train'].classes
class_names

['Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [9]:
# Enable the GPU if it exists
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
model_resnet = models.resnet18(pretrained=True)

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
model_alexnet = models.alexnet(pretrained=True)

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / data_size[phase]
            epoch_acc = running_corrects.double() / data_size[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model,best_acc


In [13]:
# Set the requires_grad on each parameter to false, 
# so it will not calculate the gradients
for param in model_resnet.parameters():
    param.requires_grad = False

for param in model_alexnet.parameters():
    param.requires_grad = False

In [14]:
# Setting the final layer
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, len(class_names))

num_ftrs = model_alexnet.classifier[6].in_features
model_alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names))

In [15]:
device

device(type='cuda', index=0)

In [16]:
# Enable GPU for the model
model_resnet = model_resnet.to(device)
model_alexnet = model_alexnet.to(device)

In [17]:
# Set the loss function
criterion = nn.CrossEntropyLoss()

In [18]:
optimizer_conv = optim.SGD(model_resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
model_resnet = train_model(model_resnet, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.8972 Acc: 0.4395
val Loss: 0.9631 Acc: 0.7253

Epoch 1/24
----------
train Loss: 1.7184 Acc: 0.5047
val Loss: 0.9327 Acc: 0.7348

Epoch 2/24
----------
train Loss: 1.7055 Acc: 0.5123
val Loss: 1.0100 Acc: 0.7164

Epoch 3/24
----------
train Loss: 1.6951 Acc: 0.5188
val Loss: 0.8442 Acc: 0.7459

Epoch 4/24
----------
train Loss: 1.6508 Acc: 0.5293
val Loss: 1.0435 Acc: 0.7330

Epoch 5/24
----------
train Loss: 1.6567 Acc: 0.5344
val Loss: 1.2452 Acc: 0.6775

Epoch 6/24
----------
train Loss: 1.6736 Acc: 0.5355
val Loss: 1.5290 Acc: 0.6760

Epoch 7/24
----------
train Loss: 1.3277 Acc: 0.6086
val Loss: 1.0117 Acc: 0.7341

Epoch 8/24
----------
train Loss: 1.2834 Acc: 0.6152
val Loss: 1.0586 Acc: 0.7183

Epoch 9/24
----------
train Loss: 1.2744 Acc: 0.6161
val Loss: 0.9277 Acc: 0.7407

Epoch 10/24
----------
train Loss: 1.2803 Acc: 0.6130
val Loss: 1.0964 Acc: 0.7128

Epoch 11/24
----------
train Loss: 1.2440 Acc: 0.6234
val Loss: 0.8729 Acc: 0.7514

Ep

In [19]:
optimizer_conv = optim.SGD(model_alexnet.classifier[6].parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
model_alexnet = train_model(model_alexnet, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 4.7547 Acc: 0.6093
val Loss: 3.1255 Acc: 0.7547

Epoch 1/24
----------
train Loss: 4.3999 Acc: 0.6760
val Loss: 4.1874 Acc: 0.7532

Epoch 2/24
----------
train Loss: 4.4422 Acc: 0.6941
val Loss: 3.2976 Acc: 0.7896

Epoch 3/24
----------
train Loss: 4.4428 Acc: 0.7008
val Loss: 3.8578 Acc: 0.7823

Epoch 4/24
----------
train Loss: 4.4508 Acc: 0.7028
val Loss: 3.6135 Acc: 0.7966

Epoch 5/24
----------
train Loss: 4.4288 Acc: 0.7118
val Loss: 4.2059 Acc: 0.7819

Epoch 6/24
----------
train Loss: 4.2884 Acc: 0.7233
val Loss: 2.5356 Acc: 0.8319

Epoch 7/24
----------
train Loss: 2.7850 Acc: 0.7771
val Loss: 2.3413 Acc: 0.8360

Epoch 8/24
----------
train Loss: 2.4980 Acc: 0.7813
val Loss: 1.8043 Acc: 0.8606

Epoch 9/24
----------
train Loss: 2.3101 Acc: 0.7889
val Loss: 1.5716 Acc: 0.8654

Epoch 10/24
----------
train Loss: 2.2600 Acc: 0.7788
val Loss: 1.6314 Acc: 0.8602

Epoch 11/24
----------
train Loss: 2.1323 Acc: 0.7874
val Loss: 1.6069 Acc: 0.8599

Ep